In [3]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [4]:
def compute_iou(annotation, mask):
    """Annotation: ground truth (512, 512), Mask: prediction (512, 512)"""

    # Compute intersection
    intersection = np.sum(np.logical_and(annotation, mask))

    # Compute union
    union = np.sum(np.logical_or(annotation, mask))

    # Compute intersection over union
    iou_score = intersection / union

    return intersection, union, iou_score

## DatasetDM

## VOC-sim

In [29]:
dataset_path = Path('voc_sim')
annotations_folder = dataset_path / 'annotations'

# Iterate throuth annotations
results = []
for annotation_path in annotations_folder.iterdir():
    example_result_dict = {}
    classname, seed, _, _, _ = annotation_path.stem.split('_')
    image_path = dataset_path / f"images/{annotation_path.stem}.jpg"
    mask_path = dataset_path / f"masks/{annotation_path.name}"
    mask_optimized_path = dataset_path / f"masks_optimized/{annotation_path.name}"
    model = "sd14dm"
    # Check all exists
    assert annotation_path.exists(), f"Annotation {annotation_path} does not exist"
    assert image_path.exists(), f"Image {image_path} does not exist"
    assert mask_path.exists(), f"Mask {mask_path} does not exist"
    assert mask_optimized_path.exists(), f"Mask {mask_optimized} does not exist"

    # Add paths to result dict
    example_result_dict['classname'] = classname
    example_result_dict['model'] = model
    example_result_dict['seed'] = seed
    example_result_dict['image_path'] = image_path.name
    example_result_dict['annotation_path'] = annotation_path.name
    example_result_dict['mask_path'] = mask_path.name

    # Load annotation. Convert in binary mask
    annotation = np.array(Image.open(annotation_path))
    assert annotation.shape == (512, 512, 3), f"Annotation {annotation_path} has wrong shape {annotation.shape}"
    annotation = annotation.sum(axis=-1) != 0
    assert annotation.shape == (512, 512), f"Annotation aggregated {annotation_path} has wrong shape {annotation.shape}"
    
    
    # Load mask (normal)
    mask = np.array(Image.open(mask_path))
    mask = mask != 0
    assert mask.shape == (512, 512), f"Mask {mask_path} has wrong shape {mask.shape}"

    i_normal, u_normal, iou_normal = compute_iou(annotation=annotation, mask=mask)
    example_result_dict['iou_normal'] = iou_normal
    example_result_dict['i_normal'] = i_normal
    example_result_dict['u_normal'] = u_normal

    # Load mask (optimized)
    mask_optimized = np.array(Image.open(mask_optimized_path))
    mask_optimized = mask_optimized != 0
    assert mask_optimized.shape == (512, 512), f"Mask optimized {mask_optimized_path} has wrong shape {mask_optimized.shape}"


    i_optimized, u_optimized, iou_optimized = compute_iou(annotation=annotation, mask=mask_optimized)
    example_result_dict['iou_optimized'] = iou_optimized
    example_result_dict['i_optimized'] = i_optimized
    example_result_dict['u_optimized'] = u_optimized

    results.append(example_result_dict)

    
# # Aggregated by example
df_results = pd.DataFrame(results)
df_results['experiment'] = "voc-sim - datasetdm"
df_results.to_csv(dataset_path / 'datasetdm_voc_sim_results.csv', index=False)

# Aggregated results by class
df_classes = df_results.groupby(['classname', 'model']).aggregate({'i_normal': 'sum', 'u_normal': 'sum', 'i_optimized': 'sum', 'u_optimized': 'sum'}).reset_index()
df_classes['iou_normal'] = df_classes['i_normal'] / df_classes['u_normal']
df_classes['iou_optimized'] = df_classes['i_optimized'] / df_classes['u_optimized']
df_classes['experiment'] = "voc-sim - datasetdm"
df_classes = df_classes.sort_values('classname').reset_index(drop=True)
df_classes.to_csv(dataset_path / 'datasetdm_voc_sim_class_results.csv', index=False)

df_overall = df_classes.groupby('model').aggregate({'i_normal': 'sum', 'u_normal': 'sum', 'i_optimized': 'sum', 'u_optimized': 'sum',
                                        'iou_normal': 'mean', 'iou_optimized': 'mean'}).reset_index()

df_overall.rename(columns={'iou_normal': 'miou_normal', 'iou_optimized': 'miou_optimized'}, inplace=True)
df_overall['iou_overall_normal'] = df_overall['i_normal'] / df_overall['u_normal']
df_overall['iou_overall_optimized'] = df_overall['i_optimized'] / df_overall['u_optimized']
df_overall['experiment'] = "voc-sim - datasetdm"
df_overall.to_csv(dataset_path / 'datasetdm_voc_sim_overall_results.csv', index=False)

df_overall_display = df_overall[["miou_normal","iou_overall_normal", "miou_optimized",  "iou_overall_optimized"]]
df_overall_display = (100*df_overall_display).round(1)
display(df_overall_display)

assert dataset_path.name == 'voc_sim', f"Dataset path {dataset_path} is not voc_sim"
df_classes_display = df_classes[['classname', 'iou_normal', 'iou_optimized']].copy()
df_classes_display['iou_normal'] = (100*df_classes_display['iou_normal']).round(1)
df_classes_display['iou_optimized'] = (100*df_classes_display['iou_optimized']).round(1)
df_classes_display.T.to_excel(dataset_path / 'datasetdm_voc_sim_class_results.xlsx', index=False)
display(df_classes_display.T)

,miou_normal,iou_overall_normal,miou_optimized,iou_overall_optimized
0,80.3,80.0,80.6,80.3


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
classname,aeroplane,bicycle,bird,boat,bottle,bus,car,cat,chair,cow,dining table,dog,horse,motorbike,person,potted plant,sheep,sofa,train,tvmonitor
iou_normal,85.0,41.1,92.5,85.5,88.1,95.5,95.1,90.1,64.1,92.8,43.6,86.7,93.4,79.5,84.2,62.4,86.2,79.1,92.1,68.4
iou_optimized,85.1,42.2,92.4,85.5,88.1,95.5,95.2,90.0,64.2,92.7,43.6,86.5,93.5,83.1,85.2,62.6,86.3,79.7,92.1,68.3


# COCO-cap

In [45]:
dataset_path = Path('coco_captions')
annotations_folder = dataset_path / 'annotations'
df_coco_captions = pd.read_csv('../coco_captions_sampled.csv')

# Iterate throuth annotations
results = []
for annotation_path in annotations_folder.iterdir():
    example_result_dict = {}
    classname, seed, _, caption, _ = annotation_path.stem.split('_')
    image_path = dataset_path / f"images/{annotation_path.stem}.jpg"
    mask_path = dataset_path / f"masks/{annotation_path.name}"
    mask_optimized_path = dataset_path / f"masks_optimized/{annotation_path.name}"
    assert annotation_path.exists(), f"Annotation {annotation_path} does not exist"
    assert image_path.exists(), f"Image {image_path} does not exist"
    assert mask_path.exists(), f"Mask {mask_path} does not exist"
    assert mask_optimized_path.exists(), f"Mask {mask_optimized_path} does not exist"
    
    # Add paths to result dict
    example_result_dict['classname'] = classname
    example_result_dict['model'] = model
    example_result_dict['seed'] = seed
    
    example_result_dict['image_path'] = image_path.name
    example_result_dict['annotation_path'] = annotation_path.name
    example_result_dict['mask_path'] = mask_path.name

    # Get info of coco caption used using caption_id
    caption_id = int(caption.replace('caption', ''))
    row = df_coco_captions.query("caption_id==@caption_id")
    assert len(row) == 1, f"Caption {caption_id} not found in df_coco_captions"
    row = row.iloc[0]
    prompt = row['caption']
    word_included = row['word_included']
    coco_categories = row['categories']

    # Add info to results
    example_result_dict['coco_caption_id'] = caption_id
    example_result_dict['prompt'] = prompt
    example_result_dict['word_included'] = word_included
    example_result_dict['coco_categories'] = coco_categories

    # Load annotation. Convert in binary mask
    annotation = np.array(Image.open(annotation_path))
    
    assert annotation.shape == (512, 512, 3), f"Annotation {annotation_path} has wrong shape {annotation.shape}"
    annotation = annotation.sum(axis=-1) != 0
    assert annotation.shape == (512, 512), f"Annotation aggregated {annotation_path} has wrong shape {annotation.shape}"
    
    # Load mask (normal)
    mask = np.array(Image.open(mask_path))
    mask = mask != 0
    assert mask.shape == (512, 512), f"Mask {mask_path} has wrong shape {mask.shape}"

    i_normal, u_normal, iou_normal = compute_iou(annotation=annotation, mask=mask)
    example_result_dict['iou_normal'] = iou_normal
    example_result_dict['i_normal'] = i_normal
    example_result_dict['u_normal'] = u_normal

    # Load mask (optimized)
    mask_optimized = np.array(Image.open(mask_optimized_path))
    mask_optimized = mask_optimized != 0
    assert mask_optimized.shape == (512, 512), f"Mask optimized {mask_optimized_path} has wrong shape {mask_optimized.shape}"
        
    i_optimized, u_optimized, iou_optimized = compute_iou(annotation=annotation, mask=mask_optimized)
    example_result_dict['iou_optimized'] = iou_optimized
    example_result_dict['i_optimized'] = i_optimized
    example_result_dict['u_optimized'] = u_optimized

    results.append(example_result_dict)    

df_results = pd.DataFrame(results)
df_results['experiment'] = "coco-cap - datasetdm"
df_results.to_csv(dataset_path / 'datasetdm_coco_captions_results.csv', index=False)


In [51]:
# All results (included and not included)
assert dataset_path.name == 'coco_captions', f"Dataset path {dataset_path} is not coco_captions"

# Aggregated results by class
df_classes = df_results.groupby(['classname', 'model']).aggregate({'i_normal': 'sum', 'u_normal': 'sum', 'i_optimized': 'sum', 'u_optimized': 'sum'}).reset_index()
df_classes['iou_normal'] = df_classes['i_normal'] / df_classes['u_normal']
df_classes['iou_optimized'] = df_classes['i_optimized'] / df_classes['u_optimized']
df_classes['experiment'] = "voc-sim - datasetdm"
df_classes = df_classes.sort_values('classname').reset_index(drop=True)
df_classes.to_csv(dataset_path / 'datasetdm_coco-cap_class_results_all.csv', index=False)

df_classes_display = df_classes[["classname", 'iou_normal', 'iou_optimized']].copy()
df_classes_display['iou_normal'] = (100*df_classes_display['iou_normal']).round(1)
df_classes_display['iou_optimized'] = (100*df_classes_display['iou_optimized']).round(1)
df_classes_display.T.to_excel(dataset_path / 'datasetdm_coco-cap_class_results_all.xlsx', index=False)

display(df_classes_display.T)

# Aggregate overall results
df_overall = df_classes.groupby('model').aggregate({'i_normal': 'sum', 'u_normal': 'sum', 'i_optimized': 'sum', 'u_optimized': 'sum',
                                       'iou_normal': 'mean', 'iou_optimized': 'mean'}).reset_index()

df_overall.rename(columns={'iou_normal': 'miou_normal', 'iou_optimized': 'miou_optimized'}, inplace=True)
df_overall['iou_overall_normal'] = df_overall['i_normal'] / df_overall['u_normal']
df_overall['iou_overall_optimized'] = df_overall['i_optimized'] / df_overall['u_optimized']
df_overall['experiment'] = "coco-cap - grounded diffusion"
df_overall.to_csv(dataset_path / 'datasetdm_coco-cap_overall_results_all.csv', index=False)

df_overall_display = df_overall[['iou_overall_normal', 'miou_normal', 'iou_overall_optimized', 'miou_optimized']].copy()
df_overall_display = (100*df_overall_display).round(1)
df_overall_display[["miou_normal","iou_overall_normal", "miou_optimized",  "iou_overall_optimized"]]



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
classname,aeroplane,bicycle,bird,boat,bottle,bus,car,cat,chair,cow,dining table,dog,horse,motorbike,person,potted plant,sheep,sofa,train,tvmonitor
iou_normal,74.1,25.7,87.4,71.3,25.5,91.4,51.9,76.2,14.8,90.1,54.6,71.7,83.5,56.4,52.2,44.6,85.8,25.6,72.9,30.1
iou_optimized,73.7,26.7,87.2,71.2,25.5,95.1,53.5,81.2,17.0,89.9,55.1,80.8,81.8,67.0,53.5,44.6,85.7,18.4,72.8,29.0


,miou_normal,iou_overall_normal,miou_optimized,iou_overall_optimized
0,59.3,59.4,60.5,60.3


In [ ]:
# All results (word_included)

assert dataset_path.name == 'coco_captions', f"Dataset path {dataset_path} is not coco_captions"

for included in [True, False]:
    included_str = 'included' if included else 'non_included'
    print(included_str)
    # Aggregated results by class
    df_classes = df_results.query("word_included==@included").groupby(['classname', 'model']).aggregate({'i_normal': 'sum', 'u_normal': 'sum', 'i_optimized': 'sum', 'u_optimized': 'sum'}).reset_index()
    df_classes['iou_normal'] = df_classes['i_normal'] / df_classes['u_normal']
    df_classes['iou_optimized'] = df_classes['i_optimized'] / df_classes['u_optimized']
    df_classes['experiment'] = "voc-sim - grounded diffusion"
    df_classes = df_classes.sort_values('classname').reset_index(drop=True)
    #df_classes.to_csv(dataset_path / 'grounded_diffusion_coco-cap_class_results_all.csv', index=False)

    df_classes_display = df_classes[["classname", 'iou_normal', 'iou_optimized']].copy()
    df_classes_display['iou_normal'] = (100*df_classes_display['iou_normal']).round(1)
    df_classes_display['iou_optimized'] = (100*df_classes_display['iou_optimized']).round(1)
    df_classes_display.T.to_excel(dataset_path / f'datasetdm_coco-cap_class_results_{included_str}.xlsx', index=False)

    display(df_classes_display.T)

    # Aggregate overall results
    df_overall = df_classes.groupby('model').aggregate({'i_normal': 'sum', 'u_normal': 'sum', 'i_optimized': 'sum', 'u_optimized': 'sum',
                                        'iou_normal': 'mean', 'iou_optimized': 'mean'}).reset_index()

    df_overall.rename(columns={'iou_normal': 'miou_normal', 'iou_optimized': 'miou_optimized'}, inplace=True)
    df_overall['iou_overall_normal'] = df_overall['i_normal'] / df_overall['u_normal']
    df_overall['iou_overall_optimized'] = df_overall['i_optimized'] / df_overall['u_optimized']
    df_overall['experiment'] = "coco-cap - grounded diffusion"
    df_overall.to_csv(dataset_path / f'datasetdm_coco-cap_overall_results_{included_str}.csv', index=False)

    df_overall_display = df_overall[['iou_overall_normal', 'miou_normal', 'iou_overall_optimized', 'miou_optimized']].copy()
    df_overall_display = (100*df_overall_display).round(1)
    display(df_overall_display)

